In [59]:
from gurobipy import Model, GRB, quicksum

In [60]:
# Create a new model
m = Model("safe_injection_clinics")

In [61]:
# Number of sites
N = 14

In [62]:
# Decision Variables
x = m.addVars(N, vtype=GRB.BINARY, name="x")
y = m.addVars(N, vtype=GRB.INTEGER, name="y")

In [63]:
# Objective: Minimize total cost
m.setObjective(quicksum(1000000 * x[n] + 100 * y[n] for n in range(N)), GRB.MINIMIZE)

In [64]:
# Constraints

# Customer assignment constraints
m.addConstr(quicksum(y[n] for n in range(N)) >= 4000, "min_customers")
m.addConstr(quicksum(y[n] for n in range(N)) <= 8000, "max_customers")
m.addConstrs((y[n] <= 600 * x[n] for n in range(N)), "max_customers_per_clinic")

# Clinic opening constraints
m.addConstr(quicksum(x[n] for n in range(N)) >= 5, "min_clinics")
m.addConstr(quicksum(x[n] for n in range(N)) <= 14, "max_clinics")

# Specific logical constraints
m.addConstr(x[6] + x[8] <= 1, "exclusive_7_9")
m.addConstr(x[1] <= x[10], "dependency_2_11")
m.addConstr(x[7] + x[9] <= 1, "exclusive_8_10")
m.addConstr(x[2] <= x[11] + x[13], "dependency_3_12_14")
m.addConstr(2 * x[0] <= x[9] + x[12], "dependency_1_10_13")

<gurobi.Constr *Awaiting Model Update*>

In [65]:
# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 28 columns and 96 nonzeros
Model fingerprint: 0x3fe0f16e
Variable types: 0 continuous, 28 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+03]
Found heuristic solution: objective 8400000.0000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 21 rows, 28 columns, 68 nonzeros
Variable types: 0 continuous, 28 integer (14 binary)

Root relaxation: objective 7.066667e+06, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7066666.67    0    1 8400000.00 7066666.67  15.9%     -   

In [66]:
# Print solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal solution found with total cost: ${m.objVal:.2f}")
    for n in range(N):
        if x[n].X > 0.5:  # Site is open
            print(f"Clinic at site {n+1} is open with {y[n].X} customers.")
else:
    print("No optimal solution found.")

Optimal solution found with total cost: $7400000.00
Clinic at site 3 is open with 600.0 customers.
Clinic at site 4 is open with 600.0 customers.
Clinic at site 5 is open with 600.0 customers.
Clinic at site 6 is open with 600.0 customers.
Clinic at site 7 is open with 600.0 customers.
Clinic at site 11 is open with 400.0 customers.
Clinic at site 12 is open with 600.0 customers.
